# 1.Write a script that connects to 100 hosts, looks for a particular process and sends an email with a report


clarifying questions
1. how do we authenticate (SSH key or password)
2. what process are we searching for ? like do we have a process name, id so i can do px aux | grep processname or pid
or lsof -i :PORT, 
1.  i assume to use SMTP to send mail directly 

Approach 1 if SSH+ grep for process + SMTP email:
i would write a python script using paramiko(sshclient + shell commands) or asyncssh+asyncio
search with pgrep or ps aux | processname
collect output , format summary and email via SMTP

In [ ]:
import paramiko  # for SSH connection
import smtplib  # for sending email via SMTP
# from email.mime.text import MIMEText  # to format email body

# List of remote hosts to connect to
hosts = ["example.com", "example2.com"]
process_name = "node"
username = "ubuntu"
keyfile = "id_rsa"

# List to store result tuples: (host, status, message)
results = []

# SSH and check process on each host
def check_process(host):
    try:
        ssh = paramiko.SSHClient()
        ssh.connect(host, username=username, key_filename=keyfile, timeout=5)

        # Run shell command to check process
        stdin, stdout, stderr = ssh.exec_command(f"ps aux | grep {process_name} | grep -v grep")
        output = stdout.read().decode().strip()

        if output:
            results.append((host, "FOUND", output))
        else:
            results.append((host, "NOT_FOUND", ""))

        ssh.close()

    except Exception as e:
        results.append((host, "ERROR", str(e)))

# Run the check on each host
for host in hosts:
    check_process(host)

# Categorize results
found = [f"{h}: {msg}" for h, status, msg in results if status == "FOUND"]
not_found = [f"{h}: Not Found" for h, status, msg in results if status == "NOT_FOUND"]
errors = [f"{h}: ERROR - {msg}" for h, status, msg in results if status == "ERROR"]

# Build report string
report = []
report.append(f"Process '{process_name}' found on {len(found)} host(s):\n" + "\n".join(found))
report.append(f"\nNot running on {len(not_found)} host(s):\n" + "\n".join(not_found))
report.append(f"\nErrors on {len(errors)} host(s):\n" + "\n".join(errors))
final_message = "\n\n".join(report)

# Send email
msg = MIMEText(final_message)
msg["Subject"] = f"[Process Report] '{process_name}' across {len(hosts)} hosts"
msg["From"] = "alerts@example.com"
msg["To"] = "admin@example.com"

with smtplib.SMTP("smtp.example.com", 587) as server:
    server.starttls()
    server.login("user", "pass")
    server.send_message(msg)


Process 'node' found on 1 host(s):
example.com: ubuntu 1234 ... /usr/bin/node app.js

Not running on 1 host(s):
example2.com: Not Found

Errors on 0 host(s):


The time and space complexity is O(N), where N is the number of hosts. For each host, I make one SSH connection, run a small process check, and store the result. At the end, I aggregate all results and send a single report. It’s efficient and works well up to a few hundred hosts — for thousands of hosts, I’d consider using threads or asyncio for concurrency

we can 
If you use threads or asyncio, the time complexity remains O(N), but wall-clock time improves to approximately O(N / P),

In [ ]:
scenario 2: based on ssh thingy and process